<a href="https://colab.research.google.com/github/KadinRelefourd/CNNandResnetClassifier/blob/main/CNNclassify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing THOP for  parameter counting

In [ ]:
!pip install thop

In [ ]:
import os
import cv2  # OpenCV for image processing
import sys
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
from thop import profile  # For counting MACs and parameters
from resnet20_cifar import resnet20  # Import the pretrained ResNet-20 model
import time


GPU Availability

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

#ranodom_seeds = [53,152,512]


Using device: cuda


Resnet Loaded

In [ ]:
# This resnet20 model is trained with Normalization augmentation for training data:
# transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
# the above transforms.Normalize(xxxx) is in the  testing data loader.
model = resnet20()
model_path = "./resnet20_cifar10_pretrained.pt"
model.load_state_dict(torch.load(model_path))



<All keys matched successfully>

Prepare Training and Testing data

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

train_data = torchvision.datasets.CIFAR10(
    root='./data.cifar10',
    train=True,
    transform=transform,
    download=True
)
train_loader = DataLoader(dataset=train_data, batch_size=64, shuffle=True)

test_data = torchvision.datasets.CIFAR10(
    root='./data.cifar10',
    train=False,
    transform=transform,
    download=True
)
test_loader = DataLoader(dataset=test_data, batch_size=64, shuffle=False)
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Build Model

In [ ]:
class Net(nn.Module):
    def __init__(self):
        # initialize parent class
        super(Net, self).__init__()  # Initialize the parent class (nn.Module)

        # create sequential model architecture
        self.model = nn.Sequential(
            # first conv layer with 32 filters
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=5, stride=1, padding=0),  # Conv layer: 3 input channels (RGB), 32 output channels, 5x5 kernel
            # apply relu activation
            nn.ReLU(),  # Apply ReLU activation function
            # reduce spatial dimensions
            nn.MaxPool2d(kernel_size=2, stride=2),  # Max pooling with 2x2 window, stride 2 (reduces spatial size by half)

            # second conv layer with 64 filters
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),  # Conv layer: 32 -> 64 channels, 3x3 kernel
            # apply relu activation
            nn.ReLU(),  # Apply ReLU activation
            # reduce spatial dimensions again
            nn.MaxPool2d(kernel_size=2, stride=2),  # Max pooling, reduces spatial dimensions

            # third conv layer maintaining 64 filters
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1),  # Another convolution layer, same 64 channels
            # apply relu activation
            nn.ReLU(),  # Apply ReLU activation

            # convert 2d feature maps to 1d vector
            nn.Flatten(),  # Flatten feature maps into a 1D tensor before passing into fully connected layers

            # first fully connected layer
            nn.Linear(64 * 7 * 7, 512),  # Fully connected layer: input size based on feature map size, output 512 neurons
            # normalize activations
            nn.BatchNorm1d(512),  # Apply batch normalization for stable training
            # prevent overfitting
            nn.Dropout(0.5),  # Dropout (50%) to reduce overfitting
            # apply relu activation
            nn.ReLU(),  # Apply ReLU activation

            # second fully connected layer
            nn.Linear(512, 256),  # Fully connected layer: 512 -> 256 neurons
            # normalize activations
            nn.BatchNorm1d(256),  # Apply batch normalization
            # prevent overfitting
            nn.Dropout(0.5),  # Dropout (50%)
            # apply relu activation
            nn.ReLU(),  # Apply ReLU activation

            # output layer for 10 classes
            nn.Linear(256, 10)  # Output layer: 10 neurons for classification (e.g., 10 classes in CIFAR-10)
        )

    def forward(self, x):
        # pass input through model layers
        return self.model(x)  # Forward pass: pass input through the model



# create model instance and move to device
model = Net().to(device)

Loss Function and Optimizer

In [ ]:
# define cross entropy loss for classification
loss_func = nn.CrossEntropyLoss()

# setup adam optimizer for training
optimizer = optim.Adam(model.parameters(), lr=0.001)

Saving Model

In [ ]:
def save():
    # Save model in the model folder
    if not os.path.exists('./model'):
        os.makedirs('./model')
    torch.save(model.state_dict(), './model/model.ckpt')
    print(f"Model saved in file: ./model/model.ckpt")

Load Model

In [ ]:
def load(model_path):
    # Load model for classification
    model.load_state_dict(torch.load(model_path))
    return model


In [ ]:
# define cross entropy loss for classification
loss_func = nn.CrossEntropyLoss()

# setup adam optimizer for training
optimizer = optim.Adam(model.parameters(), lr=0.001)

Train Accuracy

In [ ]:
def train_accuracy():
    # set model to evaluation mode
    model.eval()
    correct = 0
    total = 0

    # disable gradient calculation
    with torch.no_grad():
        for inputs, targets in train_loader:
            # get model predictions
            outputs = model(inputs)
            # get predicted class
            _, predicted = torch.max(outputs.data, 1)
            # count total samples
            total += targets.size(0)
            # count correct predictions
            correct += (predicted == targets).sum().item()

    # calculate accuracy percentage
    return 100 * correct / total

mean asnd stadard deviation

In [ ]:
"""calculate training statistics"""
def calculate_test_stats(test_accuracies):
    # convert list to numpy array
    test_accuracies = np.array(test_accuracies)

    # calculate mean accuracy
    mean_accuracy = np.mean(test_accuracies)

    # calculate standard deviation
    std_accuracy = np.std(test_accuracies)

    # print
    print(f"Mean accuracy: {mean_accuracy:.2f}%")
    print(f"Standard deviation: {std_accuracy:.2f}%")

    return mean_accuracy, std_accuracy

Test

In [ ]:
def test():
    # set model to evaluation mode
    model.eval()
    correct_test = 0
    total_test = 0
    total_loss = 0.0
    # disable gradient calculation for inference
    with torch.no_grad():
        for inputs, labels in test_loader:
            # move data to device
            inputs, labels = inputs.to(device), labels.to(device)
            # get model predictions
            outputs = model(inputs)
            # calculate loss
            loss = loss_func(outputs, labels)
            # accumulate loss
            total_loss += loss.item()
            # get predicted class
            _, predicted = outputs.max(1)
            # count total samples
            total_test += labels.size(0)
            # count correct predictions
            correct_test += predicted.eq(labels).sum().item()
    # calculate accuracy percentage
    accuracy = correct_test / total_test * 100
    # calculate average loss
    avg_loss = total_loss / len(test_loader)
    return accuracy, avg_loss


visuaiilization

In [ ]:
def visualize_conv_layer(image_path):
    # load trained model
    model = load('./model/model.ckpt')
    model.eval()

    # read input image
    img = cv2.imread(image_path)
    if img is None:
        print("Error: Image not found or invalid path.")
        return

    # convert color space
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # resize image to network input size
    resized = cv2.resize(img, (32, 32))
    # prepare tensor and move to device
    tensor = transform(resized).unsqueeze(0).to(device)

    # setup activation storage
    activation = {}
    def get_activation(name):
        def hook(model, input, output):
            # store layer output
            activation[name] = output.detach()
        return hook

    # attach hook to first conv layer
    handle = model.model[0].register_forward_hook(get_activation('conv1'))
    # perform forward pass
    with torch.no_grad():
        output = model(tensor)

    # get activation maps
    feature_maps = activation['conv1'].squeeze(0).cpu()

    # create visualization figure
    plt.figure(figsize=(10, 10))
    for i in range(32):  # 32 filters in the first conv layer
        # create subplot for each filter
        plt.subplot(6, 6, i+1)
        # display feature map
        plt.imshow(feature_maps[i], cmap='viridis')
        # hide axes
        plt.axis('off')

    # adjust layout and save
    plt.tight_layout()
    plt.savefig('CONV_rslt.png')
    plt.close()

    # cleanup hook
    handle.remove()

Train Function

In [ ]:
def train(seed=None):
    if seed is not None:
        print(f"Using random seed: {seed}")
        # set random seeds for reproducibility
        torch.manual_seed(seed)
        np.random.seed(seed)
        if torch.cuda.is_available():
            # set cuda seeds too
            torch.cuda.manual_seed(seed)
            torch.backends.cudnn.deterministic = True
            torch.backends.cudnn.benchmark = False

    # print header for training progress
    print(f"{'Epoch':<8}{'Train Loss':<15}{'Train Acc %':<15}{'Test Loss':<15}{'Test Acc %':<15}")
    train_accuracies = []
    test_accuracies = []

    for epoch in range(7):  # Train for 10 epochs
        # set model to training mode
        model.train()
        running_loss = 0.0
        correct_train = 0
        total_train = 0

        for inputs, labels in train_loader:
            # move data to device
            inputs, labels = inputs.to(device), labels.to(device)

            # reset gradients
            optimizer.zero_grad()
            # forward pass
            outputs = model(inputs)
            # calculate loss
            loss = loss_func(outputs, labels)
            # backward pass
            loss.backward()
            # update weights
            optimizer.step()

            # accumulate loss
            running_loss += loss.item()
            # get predicted class
            _, predicted = outputs.max(1)
            # count total samples
            total_train += labels.size(0)
            # count correct predictions
            correct_train += predicted.eq(labels).sum().item()

        # calculate training accuracy
        train_accuracy = correct_train / total_train * 100
        train_accuracies.append(train_accuracy)

        # evaluate on test set
        test_accuracy, test_loss = test()
        test_accuracies.append(test_accuracy)

        # print epoch results
        print(f"{epoch + 1:<8}{running_loss / len(train_loader):<15.4f}{train_accuracy:<15.2f}{test_loss:<15.4f}{test_accuracy:<15.2f}")

    # save trained model
    save()

    # create accuracy plot
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, 8), train_accuracies, label='Training Accuracy', marker='o', color='blue')
    plt.plot(range(1, 8), test_accuracies, label='Testing Accuracy', marker='s', color='red')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy (%)')
    plt.title('Training and Testing Accuracy')
    plt.legend()
    plt.grid(True)

    # save plot to file
    plot_path = "accuracy_plot.png"  # Save the plot in the current working directory
    plt.savefig(plot_path)
    plt.show()
    plt.close()

    print(f"Accuracy plot saved to: {plot_path}")

    #calculat and displau mean and stadard eviatoin
    mean_acc, std_acc = calculate_test_stats(test_accuracies)


    return test_accuracies


Test Function

In [ ]:
def classify(image_path):
    # load trained model
    model = load('./model/model.ckpt')
    model.eval()

    # read input image
    img = cv2.imread(image_path)
    if img is None:
        print("Error: Image not found or invalid path.")
        return

    # convert color space
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # resize to network input size
    resized = cv2.resize(img, (32, 32))
    # prepare tensor and move to device
    tensor = transform(resized).unsqueeze(0).to(device)

    # perform inference
    with torch.no_grad():
        output = model(tensor)
        # get predicted class
        _, predicted_class = torch.max(output.data, 1)

    # get class names
    classes = train_data.classes
    print(f'Prediction result: {classes[predicted_class.item()]}')

    # visualize activations
    ##visualize_conv_layer(image_path)


ResNeet eval funct

In [ ]:
def resnet20_eval():
    # load resnet model
    resnet_model = resnet20().to(device)
    resnet_model.load_state_dict(torch.load('./resnet20_cifar10_pretrained.pt'))

    # set to evaluation mode
    resnet_model.eval()

    correct_resnet_test = 0
    total_resnet_test = 0

    # evaluate without gradients
    with torch.no_grad():
        for inputs, labels in test_loader:
            # move data to device
            inputs, labels = inputs.to(device), labels.to(device)
            # get model predictions
            outputs_resnet = resnet_model(inputs)
            # get predicted class
            _, predicted_resnet_class = outputs_resnet.max(1)

            # count total samples
            total_resnet_test += labels.size(0)
            # count correct predictions
            correct_resnet_test += predicted_resnet_class.eq(labels).sum().item()

    # calculate accuracy percentage
    accuracy_resnet20 = correct_resnet_test / total_resnet_test * 100
    print(f"ResNet-20 Test Accuracy: {accuracy_resnet20:.2f}%")

train multiple seeds

In [ ]:
def train_multiple_seeds():
    seeds = [152, 53, 512]  # List of random seeds to use
    accuracies = []  # List to store test accuracies for each seed

    for seed in seeds:
        print(f"\nTraining with seed: {seed}")

        # Reinitialize the model and optimizer for each seed
        model = Net().to(device)  # Reinitialize the model
        optimizer = optim.Adam(model.parameters(), lr=0.001)  # Reinitialize the optimizer

        # Train the model with the current seed
        test_accuracies = train(seed=seed)  # Train with each seed
        final_test_accuracy = test_accuracies[-1]  # Get the final test accuracy
        accuracies.append(final_test_accuracy)  # Store the final accuracy

    # Calculate mean and standard deviation of the accuracies
    mean_accuracy = np.mean(accuracies)
    std_accuracy = np.std(accuracies)

    # Print the results
    print("\nTraining with multiple seeds completed.")
    print(f"Seeds used: {seeds}")
    print(f"Final test accuracies: {accuracies}")
    print(f"Mean accuracy: {mean_accuracy:.2f}%")
    print(f"Standard deviation: {std_accuracy:.2f}%")

count macs and params

In [ ]:
def count_macs_and_params(model, input_shape=(1, 3, 32, 32)):
    dummy_input = torch.randn(input_shape).to(device)
    macs, params = profile(model, inputs=(dummy_input,))
    print(f"Model: {model.__class__.__name__}")
    print(f"MACs: {macs}")
    print(f"Parameters: {params}")

inference speed test

In [ ]:
def inference_speed_test(model, iterations=100, warmup=10):
    # create dummy input
    dummy_input = torch.randn(1, 3, 32, 32).to(device)

    # set evaluation mode
    model.eval()

    # perform warmup iterations
    with torch.no_grad():
        for _ in range(warmup):
            _ = model(dummy_input)

    # measure inference time
    start_time = time.time()
    with torch.no_grad():
        for _ in range(iterations):
            # run inference
            _ = model(dummy_input)
    end_time = time.time()

    # calculate average time
    avg_time = (end_time - start_time) / iterations
    print(f"Average inference time: {avg_time:.6f} seconds")

Command Line Prompts

In [ ]:
if __name__ == "__main__":
    if len(sys.argv) < 2:
        # show usage instructions
        print("Usage:")
        print(" python CNNclassify.py train - to train CNN model")
        print(" python CNNclassify.py test <image_path> - classifies an image")
        print(" python CNNclassify.py resnet20 - evaluates ResNet-20 on CIFAR-10")
        print(" python CNNclassify.py train_multiple_seeds - tries to wiht multiple train with multiple seeds but doesn twokr")
        print(" python CNNclassify.py count_macs - count MACs and parameters of each model")
        print(" python CNNclassify.py inference_speed - tests inference speed of each model")
        sys.exit(1)

    # get command from arguments
    command = sys.argv[1]

    if command == "train":
        # train with specified seed
        train(seed=none)
        image_path = "./cat1.png"
        # visualize layer activations
        visualize_conv_layer(image_path)
    elif command == "test" and len(sys.argv) > 2:
        # classify provided image
        classify(sys.argv[2])
    elif command == "resnet20":
        # evaluate resnet performance
        resnet20_eval()
    elif command == "train_multiple_seeds":
        # train with different seeds
        train_multiple_seeds()
    elif command == "count_macs":
        # count operations in custom model
        count_macs_and_params(model)
        # load resnet model
        resnet_model = resnet20().to(device)
        resnet_model.load_state_dict(torch.load('./resnet20_cifar10_pretrained.pt'))
        # count operations in resnet
        count_macs_and_params(resnet_model)
    elif command == "inference_speed":
        # test custom model speed
        print("My Model")
        inference_speed_test(model)
        # load resnet model
        resnet_model = resnet20().to(device)
        resnet_model.load_state_dict(torch.load('./resnet20_cifar10_pretrained.pt'))
        # test resnet speed
        print("ResNet-20")
        inference_speed_test(resnet_model)
    else:
        print("Invalid command or missing parameters")

Invalid command or missing parameters


In [ ]:
!python CNNclassify.py test cat1.png

Using device: cuda
Prediction result: frog
